# Exoplanet detection from Kepler data using Spiking Neural Network


**Problem Statement**

Exoplanet detection through astronomical data has been a celebrated problem. This problem is known for its peculiar challenges for the astronomers. Advances in telescope technology have made it possible to install telescopes in space to obtain images from objects lightyears away. Thus humongous datasets have come existence which are sometimes difficult to analyse and interpret.
 
Kepler telescope was commissioned in 2009 for detection of exoplanets. Over the years the dataset obtained from the Kepler telescope has gained a reputation in astronomical data analysis similar to MNIST dataset in statistical data analysis. Since the launch of Kepler nearly all latest exoplanet discoveries are made using Artificial intelligence/data science/machine learning [AI/ML]. 

 
In recent times, state-of-the-art machine learning approaches including k-nearest neighbour, Principal Component Analysis, Convolution Neural Network, Recurrent Neural Network have been applied to the Kepler's noisy dataset. The results have been encouraging. In the year 2020 alone, 261 exoplanets were detected using ML.  However each of these methods suffer their own specific challenges. The analysis of Kepler astronomical data is  prone to high number of false positives.
 
Spiking Neural Network (SNN) is the latest addition in the line of emerging technologies proving to be a viable solution for the challenging problems in machine learning. Its performance has been noticed to improve year by year.
 
In this mini-project, I have used a deep learning neural network architecture for exoplanet planet detection from Kepler data. The dataset has been taken from Kaggle exoplanet competion dataset.

Thereafter I have converted the same architecture into a Spiking Neural Network using nengo-dl library.


**Import needed libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.optimizers.schedules import ExponentialDecay
from keras.models import load_model
from itertools import chain

pd.options.mode.chained_assignment = None  # default='warn'

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   -------- ------------------------------- 0.4/1.8 MB 8.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/1.8 MB 9.6 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.8 MB 9.3 MB/s eta 0:00:01
   --------------------------------- ------ 1.5/1.8 MB 8.8 MB/s eta 0:00:01
   --------------------------------- ------ 1.5/1.8 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 5.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/385.0 MB 7.9 MB/s eta 0:00:49
   ---------------------------------------- 0.5/385.0 MB 6.6 MB/s eta 0:00:59
   ---------------------------------------- 0.9/385.0 MB 6.9 MB/s eta 0:00:56
   ---------------------------------------- 1.4/385.0 MB 7.9 MB/s eta 0:00:49
   ---------------------------------------- 1.9/385.0 MB 8.5 MB/s eta 0:00:46
   ---------------------------------------- 2.4/385.0 MB 9.4 MB/s eta 0:00:41
   ---------------------------------------- 3.0/385.0 MB 10.0 MB/s eta 0:00:39
   ---------------------------------------- 3.7/385.0 MB 10.3 MB/s eta 0:00:38
   ---------------------------------------- 4.3/385.0 MB 10.6 MB/s eta 0:00:36
   ---------------------------------------- 4.8/385.0 MB 10.8 MB/s eta 0


### Data Preprocessing

Read the Data

In [ ]:
exoTrain = pd.read_csv('/kaggle/input/kepler-labelled-time-series-data/exoTrain.csv')
exoTest = pd.read_csv('/kaggle/input/kepler-labelled-time-series-data/exoTest.csv')

exoTrain.tail(5)

In [ ]:
exoTest.tail(5)

In [ ]:
print(exoTrain['LABEL'].value_counts())
print(exoTest['LABEL'].value_counts())

**Generate Graph of the flux values ie magnitude of the light of a particular star**

In [ ]:
def flux_graph(dataset, row, dataframe, planet):
    if dataframe:
        fig = plt.figure(figsize=(20,5))
        ax = fig.add_subplot()
        ax.set_title(planet, color='black', fontsize=22)
        ax.set_xlabel('time', color='black', fontsize=18)
        ax.set_ylabel('flux_' + str(row), color='black', fontsize=18)
        ax.grid(False)
        flux_time = list(dataset.columns)
        flux_values = dataset[flux_time].iloc[row]
        ax.plot([i + 1 for i in range(dataset.shape[1])], flux_values, 'black')
        ax.tick_params(colors = 'black', labelcolor='black', labelsize=14)
        plt.show()
    else:
        fig = plt.figure(figsize=(20,5))
        ax = fig.add_subplot()
        
        ax.set_title(planet, color='black', fontsize=22)
        ax.set_xlabel('time', color='black', fontsize=18)
        ax.set_ylabel('flux_' + str(row), color='white', fontsize=18)
        ax.grid(False)
        flux_values = dataset[row]
        ax.plot([i + 1 for i in range(dataset.shape[1])], flux_values, 'black')
        ax.tick_params(colors = 'black', labelcolor='black', labelsize=14)
        plt.show()

In [ ]:
def show_graph(dataframe, dataset):
    with_planet = exoTrain[exoTrain['LABEL'] == 2].head(3).index
    wo_planet = exoTrain[exoTrain['LABEL'] == 1].head(3).index

    for row in with_planet:
        flux_graph(dataset, row, dataframe, planet = 'periodic dip due to transiting planet')
    for row in wo_planet:
        flux_graph(dataset, row, dataframe, planet = 'no transiting planet')

In [ ]:
show_graph(True, dataset = exoTrain.loc[:, exoTrain.columns != 'LABEL'])

**Scaling**

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(exoTrain.loc[:, exoTrain.columns != 'LABEL'])
show_graph(False, scaled_data)

**Outlier handling**

In [ ]:
def handle_outliers(dataset, num_iterations):
    
    #threshold = None
    dataset_handled = dataset

    for n in range(num_iterations):
        #for column in range(dataset_handled.shape[0]):
        for index, row in dataset_handled.iterrows():
            row_values = row.values
            row_max, row_min = row_values.max(), row_values.min()
            row_maxidx, row_minidx = row_values.argmax(), row_values.argmin()
            row_mean = row_values.mean()

            #if np.abs(column_max/column_mean) >= threshold:
            dataset_handled.iloc[index][row_maxidx] = row_mean

            #if np.abs(column_min/column_mean) >= threshold:
            dataset_handled.iloc[index][row_minidx] = row_mean

    return dataset_handled

In [ ]:
handled_dataset = handle_outliers(exoTrain.loc[:, exoTrain.columns != 'LABEL'], 2)
show_graph(True, handled_dataset)

#### Apply SMOTE

In [ ]:
def lable_change(y_train, y_test):
    labler = lambda x: 1 if x == 2 else 0
    y_train_01, y_test_01 = y_train.apply(labler), y_test.apply(labler)

    return y_train_01, y_test_01

In [ ]:
def smote(x_train, y_train):
    #smote = SMOTE(random_state=17, sampling_strategy='minority')
    over = SMOTE(sampling_strategy=0.2)
    under = RandomUnderSampler(sampling_strategy=0.3)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    x_train_res, y_train_res = pipeline.fit_resample(x_train, y_train)

    return x_train_res, y_train_res

In [ ]:
y_smote_test = exoTrain.loc[:, 'LABEL']
print(y_smote_test.value_counts())
_, y_smote_test = smote(handled_dataset, y_smote_test)
print(y_smote_test.value_counts())

### Define Training and the Test Data

In [ ]:
# Define training and testing datasets
def datasets():
    x_train, y_train = exoTrain.loc[:, exoTrain.columns != 'LABEL'], exoTrain.loc[:, 'LABEL']
    x_test, y_test = exoTest.loc[:, exoTest.columns != 'LABEL'], exoTest.loc[:, 'LABEL']
    
    #fill NaNs with mean (no NaNs)
    #for column in x_train:
        #x_train[column] = x_train[column].fillna(round(x_train[column].mean(), 2))

    #x_train, x_test = scale(x_train, x_test)
    x_train = handle_outliers(x_train, 2)
    x_train, y_train = smote(x_train, y_train)
    y_train, y_test = lable_change(y_train, y_test)

    n_features = x_train.shape[1]

    return x_train, y_train, x_test, y_test, n_features

In [ ]:
# Graph train and test accuracy
def graph_acc(history):
    # Plot loss during training
    plt.subplot(211)
    plt.title('Loss')
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()

    # Plot accuracy during training
    plt.subplot(212)
    plt.title('Accuracy')
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='test')
    plt.legend()
    plt.show()

In [ ]:
# Confusion matrix
def conf_matrix(y_test, y_pred):

    matrix = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(matrix, columns=[0, 1], index = [0, 1])
    df_cm.index.name = 'Truth'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (10,7))
    sn.set(font_scale=1.4) 
    sn.heatmap(df_cm, cmap="BuGn", annot=True, annot_kws={"size": 16})
    plt.show()
    
    return matrix

In [ ]:
# Print prediction metrics
def prediction_metrics(y_test, y_pred, y_class_pred, matrix):
    FP = matrix[0][1] 
    FN = matrix[1][0]
    TP = matrix[1][1]
    TN = matrix[0][0]

    sens = TP/(TP+FN)
    spec = TN/(TN+FP) 
    g_mean = np.sqrt(sens * spec)

    accuracy = accuracy_score(y_test, y_class_pred)
    balanced_accuracy = balanced_accuracy_score(y_test, y_class_pred)
    precision = precision_score(y_test, y_class_pred)
    recall = recall_score(y_test, y_class_pred)
    f1 = f1_score(y_test, y_class_pred)
    auc = roc_auc_score(y_test, y_pred)

    print('\t\t Prediction Metrics\n')
    print("Accuracy:\t", "{:0.3f}".format(accuracy))
    print("Precision:\t", "{:0.3f}".format(precision))
    print("Recall:\t\t", "{:0.3f}".format(recall))
    print("\nF1 Score:\t", "{:0.3f}".format(f1))
    print("ROC AUC:\t", "{:0.3f}".format(auc))
    print("Balanced\nAccuracy:\t", "{:0.3f}".format(balanced_accuracy))
    print("\nSensitivity:\t", "{:0.3f}".format(sens))
    print("Specificity:\t", "{:0.3f}".format(spec))
    print("Geometric Mean:\t", "{:0.3f}".format(g_mean))

### **CNN Model**

* **Input layer**;
* **1D convolutional layer**, consisting of 10 2x2 filters, L2 regularization and RELU activation function;
* **1D max pooling layer**, window size - 2x2, stride - 2;
* **Dropout** with 20% probability;
* **Fully connected layer** with 48 neurons and RELU activation function;
* **Dropout** with 40% probability;
* **Fully connected layer** with 18 neurons and RELU activation function;
* **Output layer** with sigmoid function.

Optimizer: **Adam**  loss function: **binary-crossentropy** , **batch size**: 64 **epochs**:30 with **exponential decay** and **early stopping**.

In [ ]:
def cnn_model():

    # Data preparation
    x_train, y_train, x_test, y_test, n_features = datasets()
    x_train, y_train = shuffle(x_train, y_train) # shuffle the data to avoid stagnant 0.0000e+00 val_accuracy

    # Architecture
    model = Sequential()
    model.add(Reshape((3197, 1), input_shape=(3197,)))
    model.add(Conv1D(filters=10, kernel_size=2, activation='relu', input_shape=(n_features, 1), kernel_regularizer='l2'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(48, activation="relu"))
    model.add(Dropout(0.4))
    model.add(Dense(18, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    # Representation of architecture
    print(model.summary())

    # Compile model
    lr_schedule = ExponentialDecay(initial_learning_rate=1e-2, decay_steps=10000, decay_rate=0.94)

    model.compile(optimizer = Adam(learning_rate=lr_schedule), loss='binary_crossentropy', metrics=['accuracy'])

    # Fit model
    early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

    history = model.fit(x_train, y_train, validation_split = 0.2, batch_size=64, callbacks=[early_stop], epochs=30, verbose=2)

    # Evaluate the model
    _, train_acc = model.evaluate(x_train, y_train, verbose=2)
    _, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

    # Prediction
    y_class_pred = (model.predict(x_test) > 0.5).astype("int32")
    y_pred = model.predict(x_test)

    # Accuracy graph
    graph_acc(history)

    # Confustion matrix
    matrix = conf_matrix(y_test, y_class_pred)

    # Metrics
    prediction_metrics(y_test, y_pred, y_class_pred, matrix)
    
    return model


In [ ]:
cnn_model1=cnn_model() #For use on the conversion mudule later

**Now lets convert the CNN Model to Spiking Neural Network using Nengo-DL**

In [ ]:
from keras import layers, models

input_layer = layers.Input(batch_shape=cnn_model1.layers[0].input_shape)
prev_layer = input_layer
for layer in cnn_model1.layers:
    prev_layer = layer(prev_layer)

funcmodel = models.Model([input_layer], [prev_layer])

In [ ]:
!pip install nengo-dl #install nengo-dl if running first time
import nengo
import nengo_dl

import tensorflow as tf
#print(cnn_model1.summary())

sfr = 20
converter = nengo_dl.Converter(
    funcmodel,
    swap_activations={
        tf.keras.activations.relu: nengo.SpikingRectifiedLinear()},
    scale_firing_rates=sfr,
    synapse=0.005,
    inference_only=False)


**Define the dataset with timestep to feed into the Spiking Neural Netowrk**

In [ ]:
x_train, y_train, x_test, y_test, n_features = datasets()


print("original train ", x_train.shape,"original ytrain label ", y_train.shape)
print("original test ", x_test.shape,"original ytest label ", y_test.shape)
print("\n")

x_train, y_train, x_test, y_test = x_train.to_numpy(), y_train.to_numpy(), x_test.to_numpy(), y_test.to_numpy()

x_train = x_train.reshape((x_train.shape[0], 1, -1))
y_train = y_train.reshape((y_train.shape[0],1,-1))

x_test = x_test.reshape((x_test.shape[0], 1, -1))
y_test = y_test.reshape((y_test.shape[0],1,-1))

#an_array = np.where(an_array > 20, 0, an_array)

y_train=np.where(y_train==1,0.9,y_train)
y_test=np.where(y_test==1,0.9,y_test)

print("timestep train ", x_train.shape,"timestep train label ", y_train.shape)
print("timestep test ", x_test.shape,"timestep test label ", y_test.shape)

In [ ]:
do_training = True #Just a switch to apply training step in SNN. This is according to the documentation of nengo-dl

if do_training:
    with nengo_dl.Simulator(converter.net, minibatch_size=200) as sim:
        # run training
        sim.compile(
            optimizer=tf.optimizers.Adam(0.001),
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=[tf.metrics.sparse_categorical_accuracy],
        )
        sim.fit(
            {converter.inputs[converter.model.input]: x_train},
            {converter.outputs[converter.model.output]: y_train},
            validation_data=(
                {converter.inputs[converter.model.input]: x_test},
                {converter.outputs[converter.model.output]: y_test},
            ),
            epochs=10,
        )

        # save the parameters to file
        sim.save_params("./keras_to_snn_params")